# Get Previous Work Ranking
*DATE:* 2022-08-06

*SUMMARY:* Get previous works' ranking, rating, etc.

*FEATURE:* Use Selenium Webdriver for scraping information. 

**WARNING**\
It is based on the HTML/CSS format from August, 2022. So it may not function correctly now. For correction, check Naver Webtoon homepage's updated HTML/CSS format.

In [3]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
import pandas as pd
import time
from tqdm import tqdm
import re
from urllib.request import urlretrieve 

# 1. Load Previous Works' Information

In [4]:
df = pd.read_csv('최종_네이버웹툰_과거작품정보(2).csv')
df

,ID,Title,Author,Web Like,Genre,Age,Form,Content,Imgsrc
0,723046,하우스키퍼,채용택 / 유현,"99,999+","스토리, 판타지",15세 이용가,스크롤,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.'\nA.I 혁명이 일어난 세계...,https://shared-comic.pstatic.net/thumb/webtoon...
1,670139,부활남,채용택 / 김재한,"99,999+","스토리, 액션",15세 이용가,스크롤,나는 죽으면\n3일 만에 부활한다!,https://shared-comic.pstatic.net/thumb/webtoon...
2,703308,신석기녀,재아 / 한가람,"99,999+","스토리, 액션",12세 이용가,스크롤,불로불사 소녀와 시한부 청년의 피튀기는 로맨스,https://shared-comic.pstatic.net/thumb/webtoon...
3,754875,아이레,설이 / Team 설이,"15,061","스토리, 판타지",12세 이용가,스크롤,사라진 여왕의 힘을 찾기 위한 여정.\n그들의 위대한 모험이 시작된다.,https://shared-comic.pstatic.net/thumb/webtoon...
4,783052,퀘스트지상주의,박태준 만화회사,"1,376","스토리, 드라마",15세 이용가,스크롤,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\n공부, 싸...",https://shared-comic.pstatic.net/thumb/webtoon...
...,...,...,...,...,...,...,...,...,...
461,336945,길에서 만나다,쥬드프라이데이,"3,898","스토리, 감성",전체연령가,스크롤,서울 남산타워에서 우연히 만난 은희수와 미키의 로드무비.,https://shared-comic.pstatic.net/thumb/webtoon...
462,746740,냄새를 보는 소녀,만취,"6,983","스토리, 드라마",전체연령가,스크롤,냄새를 눈으로 보신 적 있으신가요?,https://shared-comic.pstatic.net/thumb/webtoon...
463,772235,그 해 여름,김현,71,"스토리, 로맨스",15세 이용가,스크롤,"사랑에 집착하는 희범.\n공부에만 집중하고픈 유원.\n10대의 마지막 여름, 이들과...",https://shared-comic.pstatic.net/thumb/webtoon...
464,747961,인터셉트,임주이 / 우지혜,"37,165","에피소드, 로맨스",전체연령가,스크롤,비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.\n사랑...,https://shared-comic.pstatic.net/thumb/webtoon...


In [5]:
idlist = list(df['ID'])

# 2. Get Star Rating

In [6]:
driver = webdriver.Chrome('C:/Users/gynchoi/Trap/chromedriver.exe') 
weburl = 'https://comic.naver.com/webtoon/finish'
driver.get(weburl)

C:\Users\gynchoi\AppData\Local\Temp\ipykernel_8220\1138751085.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/gynchoi/Trap/chromedriver.exe')


In [7]:
starlist = []
titlelist = []
idlist1 = []
paths1 = driver.find_elements(By.CLASS_NAME, 'rating_type > strong')
paths2 = driver.find_elements(By.CLASS_NAME, 'thumb > a')

for i in tqdm(range(len(paths1))):
    star = paths1[i].text
    starlist.append(star)
    
    id = re.findall(r'\d+', paths2[i].get_attribute('href'))
    idlist1.append(int(id[0]))

100%|██████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:40<00:00, 31.35it/s]


In [8]:
col = []
df_star = pd.DataFrame(columns=col)
df_star['ID'] = idlist1
df_star['Star'] = starlist
df_star

,ID,Star
0,767919,9.98
1,775318,9.68
2,739097,9.53
3,766626,9.84
4,738145,9.98
...,...,...
1275,20762,9.68
1276,15439,9.51
1277,22073,8.83
1278,15938,9.63


# 3. Get Mobile Likes

In [8]:
moburl = 'https://m.comic.naver.com/webtoon/finish'
driver.get(moburl)

In [9]:
likelist = []
emptylist = []

for id in tqdm(idlist):
    try:
        URL = 'https://m.comic.naver.com/webtoon/list?titleId=' + str(id)
        driver.get(URL)
        time.sleep(0.5)

        like = driver.find_elements(By.CLASS_NAME,'count_num')[0].text
        likelist.append(like)
        
    except:
        print(str(id)+': 모바일로 제공하지 않는 웹툰입니다')
        emptylist.append(id)
        likelist.append('모바일로 제공하지 않는 웹툰입니다')

  2%|█▌                                                                                | 9/466 [00:08<07:35,  1.00it/s]

783055: 모바일로 제공하지 않는 웹툰입니다


  4%|███▏                                                                             | 18/466 [00:16<06:13,  1.20it/s]

632319: 모바일로 제공하지 않는 웹툰입니다


  5%|███▉                                                                             | 23/466 [00:21<06:45,  1.09it/s]

...

 94%|███████████████████████████████████████████████████████████████████████████▎    | 439/466 [06:32<00:21,  1.28it/s]

443422: 모바일로 제공하지 않는 웹툰입니다


 96%|████████████████████████████████████████████████████████████████████████████▌   | 446/466 [06:38<00:17,  1.14it/s]

628997: 모바일로 제공하지 않는 웹툰입니다


100%|████████████████████████████████████████████████████████████████████████████████| 466/466 [06:57<00:00,  1.12it/s]


In [10]:
len(emptylist)

31

In [23]:
col = []
df_int = pd.DataFrame(columns=col)
df_int['ID'] = idlist
df_int['Mob Like'] = likelist
df_int = df_int.drop_duplicates()
df_int

,ID,Mob Like
0,723046,10만
1,670139,50만
2,703308,10만
3,754875,5만
4,783052,10만
...,...,...
461,336945,1만
462,746740,5만
463,772235,1만
464,747961,5만


# 4. Ranking
The ranking is determined based on views, num stars, and Naver's popularity index.
Num stars means how many viewers have contributed to the star rating.

In [12]:
weburl = 'https://comic.naver.com/webtoon/finish'
driver.get(weburl)

In [13]:
rankstyle = ['Ntxt_popular','Ntxt_update','Ntxt_count','Ntxt_star_rate']

In [15]:
idlist2 = []
poprank = []

count = 0
weburl = 'https://comic.naver.com/webtoon/finish'
driver.get(weburl)
rankpage = driver.find_element(By.CLASS_NAME, 'Ntxt_popular')
rankpage.click()
time.sleep(1)

paths = driver.find_elements(By.CLASS_NAME, 'thumb > a')
   
for i in tqdm(range(len(paths))):   
    id = re.findall(r'\d+', paths[i].get_attribute('href'))
    idlist2.append(int(id[0]))
    
    poprank.append(count)
    count += 1

100%|██████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:30<00:00, 41.71it/s]


In [24]:
cols = []
pop = pd.DataFrame(columns=cols)

pop['ID'] = idlist2
pop['인기순'] = poprank
pop

,ID,인기순
0,795487,0
1,731130,1
2,683496,2
3,738145,3
4,767919,4
...,...,...
1275,737378,1275
1276,768095,1276
1277,769689,1277
1278,771990,1278


In [17]:
idlist3 = []
cntrank = []

count = 0
weburl = 'https://comic.naver.com/webtoon/finish'
driver.get(weburl)
rankpage = driver.find_element(By.CLASS_NAME, 'Ntxt_count')
rankpage.click()
time.sleep(1)

paths = driver.find_elements(By.CLASS_NAME, 'thumb > a')
   
for i in tqdm(range(len(paths))):   
    id = re.findall(r'\d+', paths[i].get_attribute('href'))
    idlist3.append(int(id[0]))
    
    cntrank.append(count)
    count += 1

100%|██████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:43<00:00, 29.53it/s]


In [25]:
cnt = pd.DataFrame(columns=cols)

cnt['ID'] = idlist3
cnt['조회순'] = cntrank

cnt

,ID,조회순
0,651673,0
1,654809,1
2,609480,2
3,557672,3
4,655744,4
...,...,...
1275,791117,1275
1276,783591,1276
1277,768095,1277
1278,789989,1278


In [19]:
idlist4 = []
starrank = []

count = 0
weburl = 'https://comic.naver.com/webtoon/finish'
driver.get(weburl)
rankpage = driver.find_element(By.CLASS_NAME, 'Ntxt_star_rate')
rankpage.click()
time.sleep(1)

paths = driver.find_elements(By.CLASS_NAME, 'thumb > a')
   
for i in tqdm(range(len(paths))):   
    id = re.findall(r'\d+', paths[i].get_attribute('href'))
    idlist4.append(int(id[0]))
    
    starrank.append(count)
    count += 1

100%|██████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:35<00:00, 36.40it/s]


In [26]:
star = pd.DataFrame(columns=cols)

star['ID'] = idlist4
star['별점순'] = starrank  
star

,ID,별점순
0,718020,0
1,720121,1
2,650304,2
3,729767,3
4,700139,4
...,...,...
1275,759089,1275
1276,728061,1276
1277,686911,1277
1278,449854,1278


# 5. Create Final Dataframe

In [27]:
df2 = pd.merge(df, df_int, on = 'ID', how='inner')
df2

,ID,Title,Author,Web Like,Genre,Age,Form,Content,Imgsrc,Mob Like
0,723046,하우스키퍼,채용택 / 유현,"99,999+","스토리, 판타지",15세 이용가,스크롤,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.'\nA.I 혁명이 일어난 세계...,https://shared-comic.pstatic.net/thumb/webtoon...,10만
1,670139,부활남,채용택 / 김재한,"99,999+","스토리, 액션",15세 이용가,스크롤,나는 죽으면\n3일 만에 부활한다!,https://shared-comic.pstatic.net/thumb/webtoon...,50만
2,703308,신석기녀,재아 / 한가람,"99,999+","스토리, 액션",12세 이용가,스크롤,불로불사 소녀와 시한부 청년의 피튀기는 로맨스,https://shared-comic.pstatic.net/thumb/webtoon...,10만
3,754875,아이레,설이 / Team 설이,"15,061","스토리, 판타지",12세 이용가,스크롤,사라진 여왕의 힘을 찾기 위한 여정.\n그들의 위대한 모험이 시작된다.,https://shared-comic.pstatic.net/thumb/webtoon...,5만
4,783052,퀘스트지상주의,박태준 만화회사,"1,376","스토리, 드라마",15세 이용가,스크롤,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\n공부, 싸...",https://shared-comic.pstatic.net/thumb/webtoon...,10만
...,...,...,...,...,...,...,...,...,...,...
461,336945,길에서 만나다,쥬드프라이데이,"3,898","스토리, 감성",전체연령가,스크롤,서울 남산타워에서 우연히 만난 은희수와 미키의 로드무비.,https://shared-comic.pstatic.net/thumb/webtoon...,1만
462,746740,냄새를 보는 소녀,만취,"6,983","스토리, 드라마",전체연령가,스크롤,냄새를 눈으로 보신 적 있으신가요?,https://shared-comic.pstatic.net/thumb/webtoon...,5만
463,772235,그 해 여름,김현,71,"스토리, 로맨스",15세 이용가,스크롤,"사랑에 집착하는 희범.\n공부에만 집중하고픈 유원.\n10대의 마지막 여름, 이들과...",https://shared-comic.pstatic.net/thumb/webtoon...,1만
464,747961,인터셉트,임주이 / 우지혜,"37,165","에피소드, 로맨스",전체연령가,스크롤,비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.\n사랑...,https://shared-comic.pstatic.net/thumb/webtoon...,5만


In [28]:
# Remove toons that have no ranking.
df2 = pd.merge(df2, df_star, on = 'ID')
df2

,ID,Title,Author,Web Like,Genre,Age,Form,Content,Imgsrc,Mob Like,Star
0,723046,하우스키퍼,채용택 / 유현,"99,999+","스토리, 판타지",15세 이용가,스크롤,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.'\nA.I 혁명이 일어난 세계...,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97
1,670139,부활남,채용택 / 김재한,"99,999+","스토리, 액션",15세 이용가,스크롤,나는 죽으면\n3일 만에 부활한다!,https://shared-comic.pstatic.net/thumb/webtoon...,50만,9.51
2,703308,신석기녀,재아 / 한가람,"99,999+","스토리, 액션",12세 이용가,스크롤,불로불사 소녀와 시한부 청년의 피튀기는 로맨스,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97
3,754875,아이레,설이 / Team 설이,"15,061","스토리, 판타지",12세 이용가,스크롤,사라진 여왕의 힘을 찾기 위한 여정.\n그들의 위대한 모험이 시작된다.,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.83
4,783055,신림/남/22,박태준 만화회사,388,"스토리, 스릴러",18세 이용가,컷툰,"""처음부터 들어가서는 안됐다.""\n\n외로워서 '익명채팅'을 시작한 '혁'은\n낯선...",https://shared-comic.pstatic.net/thumb/webtoon...,모바일로 제공하지 않는 웹툰입니다,5.54
...,...,...,...,...,...,...,...,...,...,...,...
341,336945,길에서 만나다,쥬드프라이데이,"3,898","스토리, 감성",전체연령가,스크롤,서울 남산타워에서 우연히 만난 은희수와 미키의 로드무비.,https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.76
342,746740,냄새를 보는 소녀,만취,"6,983","스토리, 드라마",전체연령가,스크롤,냄새를 눈으로 보신 적 있으신가요?,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.97
343,772235,그 해 여름,김현,71,"스토리, 로맨스",15세 이용가,스크롤,"사랑에 집착하는 희범.\n공부에만 집중하고픈 유원.\n10대의 마지막 여름, 이들과...",https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.94
344,747961,인터셉트,임주이 / 우지혜,"37,165","에피소드, 로맨스",전체연령가,스크롤,비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.\n사랑...,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.94


In [29]:
df2 = pd.merge(df2, pop, on = 'ID')
df2

,ID,Title,Author,Web Like,Genre,Age,Form,Content,Imgsrc,Mob Like,Star,인기순
0,723046,하우스키퍼,채용택 / 유현,"99,999+","스토리, 판타지",15세 이용가,스크롤,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.'\nA.I 혁명이 일어난 세계...,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97,57
1,670139,부활남,채용택 / 김재한,"99,999+","스토리, 액션",15세 이용가,스크롤,나는 죽으면\n3일 만에 부활한다!,https://shared-comic.pstatic.net/thumb/webtoon...,50만,9.51,271
2,703308,신석기녀,재아 / 한가람,"99,999+","스토리, 액션",12세 이용가,스크롤,불로불사 소녀와 시한부 청년의 피튀기는 로맨스,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97,239
3,754875,아이레,설이 / Team 설이,"15,061","스토리, 판타지",12세 이용가,스크롤,사라진 여왕의 힘을 찾기 위한 여정.\n그들의 위대한 모험이 시작된다.,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.83,129
4,783055,신림/남/22,박태준 만화회사,388,"스토리, 스릴러",18세 이용가,컷툰,"""처음부터 들어가서는 안됐다.""\n\n외로워서 '익명채팅'을 시작한 '혁'은\n낯선...",https://shared-comic.pstatic.net/thumb/webtoon...,모바일로 제공하지 않는 웹툰입니다,5.54,35
...,...,...,...,...,...,...,...,...,...,...,...,...
341,336945,길에서 만나다,쥬드프라이데이,"3,898","스토리, 감성",전체연령가,스크롤,서울 남산타워에서 우연히 만난 은희수와 미키의 로드무비.,https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.76,1018
342,746740,냄새를 보는 소녀,만취,"6,983","스토리, 드라마",전체연령가,스크롤,냄새를 눈으로 보신 적 있으신가요?,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.97,884
343,772235,그 해 여름,김현,71,"스토리, 로맨스",15세 이용가,스크롤,"사랑에 집착하는 희범.\n공부에만 집중하고픈 유원.\n10대의 마지막 여름, 이들과...",https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.94,900
344,747961,인터셉트,임주이 / 우지혜,"37,165","에피소드, 로맨스",전체연령가,스크롤,비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.\n사랑...,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.94,896


In [30]:
df2 = pd.merge(df2, cnt, on = 'ID')
df2

,ID,Title,Author,Web Like,Genre,Age,Form,Content,Imgsrc,Mob Like,Star,인기순,조회순
0,723046,하우스키퍼,채용택 / 유현,"99,999+","스토리, 판타지",15세 이용가,스크롤,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.'\nA.I 혁명이 일어난 세계...,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97,57,516
1,670139,부활남,채용택 / 김재한,"99,999+","스토리, 액션",15세 이용가,스크롤,나는 죽으면\n3일 만에 부활한다!,https://shared-comic.pstatic.net/thumb/webtoon...,50만,9.51,271,54
2,703308,신석기녀,재아 / 한가람,"99,999+","스토리, 액션",12세 이용가,스크롤,불로불사 소녀와 시한부 청년의 피튀기는 로맨스,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97,239,145
3,754875,아이레,설이 / Team 설이,"15,061","스토리, 판타지",12세 이용가,스크롤,사라진 여왕의 힘을 찾기 위한 여정.\n그들의 위대한 모험이 시작된다.,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.83,129,774
4,783055,신림/남/22,박태준 만화회사,388,"스토리, 스릴러",18세 이용가,컷툰,"""처음부터 들어가서는 안됐다.""\n\n외로워서 '익명채팅'을 시작한 '혁'은\n낯선...",https://shared-comic.pstatic.net/thumb/webtoon...,모바일로 제공하지 않는 웹툰입니다,5.54,35,440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,336945,길에서 만나다,쥬드프라이데이,"3,898","스토리, 감성",전체연령가,스크롤,서울 남산타워에서 우연히 만난 은희수와 미키의 로드무비.,https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.76,1018,1109
342,746740,냄새를 보는 소녀,만취,"6,983","스토리, 드라마",전체연령가,스크롤,냄새를 눈으로 보신 적 있으신가요?,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.97,884,1182
343,772235,그 해 여름,김현,71,"스토리, 로맨스",15세 이용가,스크롤,"사랑에 집착하는 희범.\n공부에만 집중하고픈 유원.\n10대의 마지막 여름, 이들과...",https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.94,900,1170
344,747961,인터셉트,임주이 / 우지혜,"37,165","에피소드, 로맨스",전체연령가,스크롤,비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.\n사랑...,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.94,896,822


In [31]:
df2 = pd.merge(df2, star, on = 'ID')
df2

,ID,Title,Author,Web Like,Genre,Age,Form,Content,Imgsrc,Mob Like,Star,인기순,조회순,별점순
0,723046,하우스키퍼,채용택 / 유현,"99,999+","스토리, 판타지",15세 이용가,스크롤,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.'\nA.I 혁명이 일어난 세계...,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97,57,516,179
1,670139,부활남,채용택 / 김재한,"99,999+","스토리, 액션",15세 이용가,스크롤,나는 죽으면\n3일 만에 부활한다!,https://shared-comic.pstatic.net/thumb/webtoon...,50만,9.51,271,54,1110
2,703308,신석기녀,재아 / 한가람,"99,999+","스토리, 액션",12세 이용가,스크롤,불로불사 소녀와 시한부 청년의 피튀기는 로맨스,https://shared-comic.pstatic.net/thumb/webtoon...,10만,9.97,239,145,133
3,754875,아이레,설이 / Team 설이,"15,061","스토리, 판타지",12세 이용가,스크롤,사라진 여왕의 힘을 찾기 위한 여정.\n그들의 위대한 모험이 시작된다.,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.83,129,774,825
4,783055,신림/남/22,박태준 만화회사,388,"스토리, 스릴러",18세 이용가,컷툰,"""처음부터 들어가서는 안됐다.""\n\n외로워서 '익명채팅'을 시작한 '혁'은\n낯선...",https://shared-comic.pstatic.net/thumb/webtoon...,모바일로 제공하지 않는 웹툰입니다,5.54,35,440,1271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,336945,길에서 만나다,쥬드프라이데이,"3,898","스토리, 감성",전체연령가,스크롤,서울 남산타워에서 우연히 만난 은희수와 미키의 로드무비.,https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.76,1018,1109,955
342,746740,냄새를 보는 소녀,만취,"6,983","스토리, 드라마",전체연령가,스크롤,냄새를 눈으로 보신 적 있으신가요?,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.97,884,1182,180
343,772235,그 해 여름,김현,71,"스토리, 로맨스",15세 이용가,스크롤,"사랑에 집착하는 희범.\n공부에만 집중하고픈 유원.\n10대의 마지막 여름, 이들과...",https://shared-comic.pstatic.net/thumb/webtoon...,1만,9.94,900,1170,426
344,747961,인터셉트,임주이 / 우지혜,"37,165","에피소드, 로맨스",전체연령가,스크롤,비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.\n사랑...,https://shared-comic.pstatic.net/thumb/webtoon...,5만,9.94,896,822,430


In [33]:
# Save as CSV format
df2.to_csv('최종_네이버웹툰_과거정보(3)종합.csv', encoding = 'utf-8-sig')